# Python notebook for post-processing apical responses.
# Create image stack movie with regions plot.
Assumes folder directory structure:
<pre><code>  IMAGING
    image_stacks
    notebooks
    results
</code></pre>
Execute the code sequentially, one block at a time, using &lt;shift-return&gt;.

In [1]:
%matplotlib widget

import glob
import ipywidgets as widgets
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os
from skimage import io
from skimage.color import gray2rgb
from skimage.feature import canny
from skimage.util import img_as_ubyte


#### Select an image stack, a results directory and one or more regions.
Select multiple regions using command-click.

In [2]:
# global variables
image_sel = ""    # the selected image stack
results_sel = ""  # the selected results directory
regions_sel = []  # a list of the region files selected
tag_inds = {}     # empty dictionary of tag indices per results directory

# create image files widget
image_files = sorted([f.split('/')[-1] for f in glob.glob("../image_stacks/*.tif", recursive=False)], key=str.casefold)
image_widget = widgets.Select(options=image_files, description='Image stack:', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# create results directory widget
result_dirs = sorted([f.split('/')[-2] for f in glob.glob("../results/*/", recursive=False)], key=str.casefold)
results_widget = widgets.Select(options=[], description='Results dir:', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# create regions widget
regions_widget = widgets.SelectMultiple(options=[], description='Regions:', 
                            disabled=False, layout=widgets.Layout(width='400px'))

# update the results dirs based on the image file selection
def update_result_dirs(*args):
  global tag_inds
  rdirs = []
  tag_inds = {}
  for d in result_dirs:
    f = open("../results/" + d + "/image_stacks.txt", 'r')
    for idx,l in enumerate(f):
      if l.rstrip() == image_widget.value:
        rdirs.append(d)
        print(d, idx)
        tag_inds.update({d:idx})
        break;
    f.close()
  results_widget.options = rdirs    # populate the results widget
  if not rdirs:                     # clear the widget if no regions
    regions_widget.options = []

# update the regions based on the results directory selection
def update_regions(*args):
  if results_widget.value == None:
    return
  region_files = os.listdir("../results/" + results_widget.value)
  region_files = sorted([f for f in region_files if 'apical_region' in f and '.csv' in f][:-1])
  #regions = [f.split('-')[0].split('_')[-1] for f in region_files]
  #regions = sorted([int(r) for r in regions])
  regions_widget.options = region_files

# widget change callbacks
image_widget.observe(update_result_dirs, 'value')
results_widget.observe(update_regions, 'value')

# display and respond to the widgets
update_result_dirs()
update_regions()
def f(w1, w2, w3):
  global image_sel, results_sel, regions_sel
  image_sel = image_widget.value
  results_sel = results_widget.value
  regions_sel = list(regions_widget.value)
display(widgets.interactive(f, w1=image_widget, w2=results_widget, w3=regions_widget))

interactive(children=(Select(description='Image stack:', layout=Layout(width='400px'), options=('lessmovement3…

#### Check that your selections are valid.

In [4]:
#print(image_sel)
#print(results_sel)
#print(regions_sel)
#print(tag_inds)

if not image_sel:
  print("ERROR: No image stack selected.")
elif not results_sel:
  print("ERROR: No result directory selected.")
elif not regions_sel:
  print("ERROR: No region(s) selected.")
else:
  print("Selection OK.")


Selection OK.


#### Generate the movie file.
NOTE: May take a minute or two to run depending on the number of regions selected.

In [5]:
backend = mpl.get_backend()
mpl.use("Agg")  # don't display the animation
print("Processing frame: ", end = '')

# get the default plot colors
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# get the image stack
image_file = "../image_stacks/" + image_sel
image_bits = 10
A0 = io.imread(image_file)
A = np.float32(A0/(2.0**image_bits))
for i in range(2,A.shape[0]-2):      # use moving average over five frames
  A[i] = np.mean(A[i-2:i+3], axis=0)
for i in range(2):                   # duplicate beginning and ending frames
  A[i] = A[2]
  A[-(1+i)] = A[-3]
A = gray2rgb(A)

# get the apical mask
M = io.imread(glob.glob("../results/" + results_sel + "/*apical_mask_labelled.tif", recursive=False)[0])

# get the results data and color the region perimeters in the image stack
D = []
for idx,s in enumerate(regions_sel):
  data_file = "../results/" + results_sel + '/' + s
  D.append(np.genfromtxt(data_file, delimiter=','))
  temp = np.zeros(A0[0].shape)                                  # this region mask
  temp[np.where(M==int(s.split('-')[0].split('_')[-1]))] = 1.0  # find this region value
  perimeter = canny(img_as_ubyte(temp))                         # create a perimeter for this region
  for im in A:                                                  # draw the perimeter in every image frame
    im[perimeter] = mpl.colors.to_rgb(colors[idx])              #    using the same colors as the plot(s)
D = np.array(D)  # convert list to numpy array
tag_ind = tag_inds[results_sel] # get the tag index for this results directory

# setup the figure
fig = plt.figure(constrained_layout=True, figsize = [12,6], dpi=100)
gs = fig.add_gridspec(2,2)
ax0 = fig.add_subplot(gs[:,0])  # the image stack
ax1 = fig.add_subplot(gs[1,1])  # the region plot(s)
ax2 = fig.add_subplot(gs[0,1])  # blank space padding
ax2.axis('off')

# image subplot
ax0.set_xticks([]) # don't show ticks or labels on the image
ax0.set_yticks([])
im = ax0.imshow(A[0], norm=None)

# region plot(s) subplot
ax1.set_xlim(0,10.0*np.ceil(D[0,-1,0]/10.0))     # just use final value in first region
ax1.set_ylim(0,100.0*np.ceil(np.amax(D[:,:,tag_ind+1]/100.0)))  # max value across all regions for tag
ax1.set(ylabel="%F/F0")
ax1.set(xlabel="time (s)")
X = [None] * len(regions_sel)     # list of plot value lists (per region)
Y = [None] * len(regions_sel)     #
lines = [None] * len(regions_sel) # list of plotted lines (per region)
for idx,r in enumerate(D):        # plot the first value in each region
  X[idx] = [r[0,0]]
  Y[idx] = [r[0,tag_ind+1]]
  line, = ax1.plot(X[idx],Y[idx])
  lines[idx] = line

# animate the figure
def animation_frame(i):
  for idx,r in enumerate(D):    # update region plot(s)
    X[idx].append(r[i,0])
    Y[idx].append(r[i,tag_ind+1])  
    lines[idx].set_xdata(X[idx])
    lines[idx].set_ydata(Y[idx])
  im.set_data(A[i])             # get current image frame
  if i:                         # progress indication
    print(str(i) + ", ", end = '')
  return
animation = FuncAnimation(fig, animation_frame, blit=False, repeat=False,
                          frames=D.shape[1], interval=np.int(1000.0*D[0,1,0]))

# save the animation to a movie file
animation.save("../results/" + results_sel + "/apical.mp4")

print("DONE.")
mpl.use(backend) # restore the backend for display


Processing frame: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 